In [1]:
from typing import Any, Dict, Optional

from langchain.agents.agent import AgentExecutor
from langchain.agents.mrkl.base import ZeroShotAgent
from langchain.base_language import BaseLanguageModel
from langchain.callbacks.base import BaseCallbackManager
from langchain.chains.llm import LLMChain
from langchain.tools.python.tool import PythonREPLTool
from langchain.llms.openai import OpenAI
from langchain.agents.agent_types import AgentType
from getpass import getpass

In [2]:
openai_api_key = getpass()

In [12]:
from langchain.tools.file_management import (
    ReadFileTool,
    CopyFileTool,
    DeleteFileTool,
    MoveFileTool,
    WriteFileTool,
    ListDirectoryTool,
)
from langchain.agents.agent_toolkits import FileManagementToolkit
from tempfile import TemporaryDirectory

# We'll make a temporary directory to avoid clutter
working_directory = TemporaryDirectory()

def create_python_agent(
    llm: BaseLanguageModel,
    tool: PythonREPLTool,
    prefix: str,
    suffix: str,
    format_instructions: str,
    callback_manager: Optional[BaseCallbackManager] = None,
    verbose: bool = False,
    agent_executor_kwargs: Optional[Dict[str, Any]] = None,
    **kwargs: Dict[str, Any],
) -> AgentExecutor:
    """Construct a python agent from an LLM and tool."""
    tools = [tool]
    tools +=  FileManagementToolkit(
        root_dir=str(working_directory.name),
        selected_tools=["write_file"],
    ).get_tools()
    
    prompt = ZeroShotAgent.create_prompt(tools, prefix=prefix, suffix=suffix, format_instructions=format_instructions)
    llm_chain = LLMChain(
        llm=llm,
        prompt=prompt,
        callback_manager=callback_manager,
    )
    tool_names = [tool.name for tool in tools]
    agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names, **kwargs)
    return AgentExecutor.from_agent_and_tools(
        agent=agent,
        tools=tools,
        callback_manager=callback_manager,
        verbose=verbose,
        **(agent_executor_kwargs or {}),
    )


In [16]:
prefix = '''You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code.
If you get an error, debug your code and try again.
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.

You then must write the code to a file called "dynamic.py".

You will start with the following code:
``` python
import streamlit as st
import pandas as pd

def dynamic(uploaded_file: pd.DataFrame):
    """
    this function will be replaced by the code generated by the LLM after the prompt of the user
    uploaded_file: is a panda dataframe pd.DataFrame
    """
    st.write("This content will be replaced")
```
'''

format_instructions = """Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do next and remember what you have done
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question which is the code to be executed"""

suffix = """Begin!

Question: {input}
Thought:{agent_scratchpad}"""


In [17]:
agent_executor = create_python_agent(
    llm=OpenAI(temperature=0, max_tokens=1000, openai_api_key=openai_api_key),
    tool=PythonREPLTool(),
    verbose=True,
    suffix=suffix,
    prefix=prefix,
    format_instructions=format_instructions,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [18]:
agent_executor.run("Add a title with the text 'This is the Title' and a write a the text 'Yo, I am a text'.")



> Entering new AgentExecutor chain...
 I need to use the streamlit library to add a title and write a text.
Action: Python REPL
Action Input: st.title('This is the Title')
st.write('Yo, I am a text')
Observation: NameError("name 'st' is not defined")
Thought: I need to import the streamlit library.
Action: Python REPL
Action Input: import streamlit as st
Observation: 
Thought: Now I can add the title and write the text.
Action: Python REPL
Action Input: st.title('This is the Title')
st.write('Yo, I am a text')
Observation: 
Thought: I now know the final answer.
Final Answer: st.title('This is the Title')
st.write('Yo, I am a text')

> Finished chain.


"st.title('This is the Title')\nst.write('Yo, I am a text')"

In [13]:
agent_executor.run("Change the title to 'This is the new title'.")



> Entering new AgentExecutor chain...
 I need to use the string method title()
Action: Python REPL
Action Input: "This is the new title".title()
Observation: SyntaxError('unterminated string literal (detected at line 1)', ('<string>', 1, 22, 'This is the new title".title()', 1, 22))
Thought: I need to add quotes around the string
Action: Python REPL
Action Input: "'This is the new title'".title()
Observation: SyntaxError('unterminated string literal (detected at line 1)', ('<string>', 1, 24, '\'This is the new title\'".title()', 1, 24))
Thought: I need to add an extra set of quotes
Action: Python REPL
Action Input: "''This is the new title''".title()
Observation: SyntaxError('unterminated string literal (detected at line 1)', ('<string>', 1, 26, '\'\'This is the new title\'\'".title()', 1, 26))
Thought:

KeyboardInterrupt: 